#ANOMALY CLASSIFICATION IN IIoT SYSTEMS

In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras import layers
from matplotlib import pyplot as plt

In [ ]:
# Detect hardware
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print(f'Running on a TPU w/{tpu.num_accelerators()["TPU"]} cores')
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime;')
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pd.options.mode.copy_on_write = True #to save on memory and avoid making copies of data until written to
df = pd.read_csv('/content/drive/MyDrive/Colab/UdeA/normalized/edge1/edge1_11.csv')
df.reindex(['localtime', 'indoor_temp', 'indoor_humidity', 'edge_soc_temp', 'edge_load1', 'edge_load15',
          'delta_sent', 'delta_recv', 'edge_current', 'motor_sound', 'motor_freq', 'motor_volt',
          'motor_current', 'motor_recipe','edge_cpu_freq', 'edge_load5','edge_cpu_voltage', 'edge_memory_free',
          'edge_memory_avail',  'edge_wifi_send', 'edge_wifi_receiv', 'fault_type', 'attack_type', 'anomaly',
          'normal', 'E', 'A', 'F', 'N'], axis=1)
x_train=df.iloc[:,3:11]
y_train = df.iloc[:,-3:]
print('y_train: ',y_train.shape)
print('x_train: ',x_train.shape)

In [7]:
# Generate training sequence
def create_sequences(values, n_samples, n_times, n_features):
  output = np.zeros( (n_samples-n_times, n_times, n_features) )
  for j in range(np.array(n_features)):
    features = []
    for i in range(np.array(n_samples - n_times)):
      output[i,:,j]=values[i : (i + n_times),j]
  return output

In [8]:
N_TIMES = 300

In [ ]:
# create_sequences(df_training, N_SAMPLES, N_TIMES, N_FEATURES) for x_train
df_training = np.array(x_train[:])
N_SAMPLES= df_training.shape[0]
N_FEATURES = df_training.shape[1]
x_train_sequence= create_sequences(np.array(df_training), np.array(N_SAMPLES), np.array(N_TIMES), np.array(N_FEATURES))
x_train_sequence = x_train_sequence.astype(np.float32)
x_train_sequence.shape

In [ ]:
# create_sequences(df_training, N_SAMPLES, N_TIMES, N_FEATURES) for y_train
df_training = np.array(y_train[:])
N_SAMPLES= df_training.shape[0]
N_FEATURES = df_training.shape[1]
y_train_sequence= create_sequences(df_training, N_SAMPLES, N_TIMES, N_FEATURES)
y_train_sequence = y_train_sequence.astype(np.float32)
y_train_sequence.shape

In [ ]:
import random
#BALANCE USING SEQUENCE ON OUTPUT
secuencia = np.sum(y_train_sequence, axis=1)
lista = list(sum(secuencia)/N_TIMES)
if lista[0] > lista[1]:
  umbral = int(lista[2] - lista[0])
else:
  umbral = int(lista[2] - lista[1])
#'umbral' has the amount of normal data that exceeds the abnormal data
print(umbral)
indices=list(np.where((secuencia[:, 2]> secuencia[:, 1]) & (secuencia[:, 2]> secuencia[:, 0]))[0])
posiciones = sorted(random.sample(indices, umbral))
#Eliminate these rows:
y_train_sequence = np.delete(y_train_sequence, posiciones, axis=0)
x_train_sequence = np.delete(x_train_sequence, posiciones, axis=0)
print(y_train_sequence.shape)
print(x_train_sequence.shape)

In [12]:
from keras.layers import MultiHeadAttention, Input, Dense, Softmax, Conv1D, Dropout, GlobalAveragePooling1D
from keras.layers import LayerNormalization, Layer
from keras.layers import TextVectorization, Embedding
from tensorflow import convert_to_tensor
from keras import Model, Sequential
import math
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [13]:
# https://colab.research.google.com/drive/1lru2zGbF3kB5N5Blu2ZXh-9Z8sxE3SSb#scrollTo=1H5Wk08rWKAj
# total_heads: How many Attention units are we using?
# dense_units:  Number of neurons in dense layer with relu-activation (you can try different values)
# embed_dim: x_train_sequence.shape[2] how many variables are there in dataset?
# sequence_length: x_train_sequence.shape[1] samples in time-window
# classes: 1: Normal - Failure - Attack

keras.saving.get_custom_objects().clear()
@keras.saving.register_keras_serializable(package="MyLayers")
class EncoderLayer(tf.keras.Model):
    def __init__(self, total_heads, dense_units, sequence_length, embed_dim, classes, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.total_heads = total_heads
        self.dense_units = dense_units
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim
        self.classes = classes
        self.multihead = MultiHeadAttention(num_heads=total_heads, key_dim=embed_dim)
        self.forward = Sequential([Dense(self.embed_dim)])
        self.normalize_layer = LayerNormalization()

    def call(self, x):
        multihead_layer = self.multihead(x,x)
        normalize1_layer = self.normalize_layer(x + multihead_layer)
        forward_layer = self.forward(normalize1_layer)
        normalize2_layer = self.normalize_layer(normalize1_layer + forward_layer)
        final_output = self.forward(normalize2_layer)
        return final_output

    def get_config(self):
        base_config = super().get_config()
        return {"total_heads": self.total_heads,"dense_units": self.dense_units,
                "sequence_length": self.sequence_length,"embed_dim": self.embed_dim,
                "classes": self.classes, **base_config}
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
# Construct the transformer model
with tpu_strategy.scope():
  total_heads=20
  sequence_length = x_train_sequence.shape[1] #timestep
  embed_dim = x_train_sequence.shape[2]       #variables in dataset
  classes = y_train_sequence.shape[2]         # A-F-N
  dense_units = embed_dim
  # Custom layers
  encoder_layer = EncoderLayer(total_heads, dense_units, sequence_length, embed_dim, classes)
  classify_layer = Dense(classes, activation='softmax')
  # Start connecting the layers together
  inputs = Input(shape=(sequence_length,embed_dim ))
  encoder_1 = encoder_layer(inputs) #directly assign x without embedding
  encoder_2 = encoder_layer(encoder_1)
  encoder_3 = encoder_layer(encoder_2)
  dense = Dense(dense_units, activation="relu")(encoder_3)
  outputs = classify_layer(dense)
  model = Model(inputs=inputs, outputs=outputs)

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy')
  model.save("/content/drive/MyDrive/Colab/models/transformer.keras")
model.summary()

In [17]:
files_dic1 = {'/content/drive/MyDrive/Colab/UdeA/normalized/edge1/': ['edge1_11.csv', 'edge1_12.csv', 'edge1_13.csv', 'edge1_14.csv']}
files_dic2 = {'/content/drive/MyDrive/Colab/UdeA/normalized/edge2/': ['edge2_9.csv','edge2_10.csv', 'edge2_11.csv', 'edge2_12.csv', 'edge2_13.csv']}
files_dic3 = {'/content/drive/MyDrive/Colab/UdeA/normalized/edge3/': ['edge3_8.csv', 'edge3_9.csv', 'edge3_10.csv', 'edge3_11.csv', 'edge3_12.csv']}
files_dic4 = {'/content/drive/MyDrive/Colab/UdeA/normalized/edge4/': ['edge4_10.csv','edge4_11.csv', 'edge4_12.csv']}
files_dic = [files_dic1, files_dic2, files_dic3, files_dic4]

In [ ]:
from pathlib import Path
datos =[]
for m in range(len(files_dic)):
  for key,value in files_dic[m].items():
    for file in value:
      my_file =key + file
      datos.append(my_file)
print(datos)
datos_val=datos[:]

**Training the model with each csv file**

As the model is stored in Google Drive, when the history is graphed, you can choose the version (epoch) with the best metrics and return to that version in Google.

In [ ]:
from pathlib import Path
import random
import json
import random
random.seed(123)
N_TIMES = 300
ciclos = 0
with tpu_strategy.scope():
  model = keras.models.load_model("/content/drive/MyDrive/Colab/models/transformer.keras", compile=False)
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy')
for i in range(50):
  for jx in range(len(datos)):
    print(datos[jx])
    df_train = pd.read_csv(datos[jx])
    x_train = df_train[['edge_soc_temp', 'edge_load1', 'edge_load15', 'delta_sent', 'delta_recv', 'edge_current', 'motor_sound', 'motor_freq']]
    x_train = x_train.to_numpy(np.float32)
    y_train = df_train.iloc[:,-3:]
    y_train = y_train.to_numpy(np.float32)

    with tpu_strategy.scope():
      #creating sequences
      N_SAMPLES= x_train.shape[0]
      N_FEATURES = x_train.shape[1]
      x_train_sequence= create_sequences(x_train, N_SAMPLES, N_TIMES, N_FEATURES)
      x_train_sequence = x_train_sequence.astype(np.float32)
      N_SAMPLES= y_train.shape[0]
      N_FEATURES = y_train.shape[1]
      y_train_sequence= create_sequences(y_train, N_SAMPLES, N_TIMES, N_FEATURES)
      y_train_sequence = y_train_sequence.astype(np.float32)
      secuencia = np.sum(y_train_sequence, axis=1)
      #eliminating normal sequences to balance data
      lista = list(sum(secuencia)/N_TIMES)
      if lista[0] > lista[1]:
        umbral = int(lista[2] - lista[0])
      else:
        umbral = int(lista[2] - lista[1])
      indices=list(np.where((secuencia[:, 2]> secuencia[:, 1]) & (secuencia[:, 2]> secuencia[:, 0]))[0])
      posiciones = sorted(random.sample(indices, umbral))
      y_train_sequence = np.delete(y_train_sequence, posiciones, axis=0)
      x_train_sequence = np.delete(x_train_sequence, posiciones, axis=0)
      history = model.fit(
          x_train_sequence,
          y_train_sequence,
          epochs = 1,
          batch_size=128,
          validation_batch_size=128,
          shuffle=False,
          callbacks=[keras.callbacks.EarlyStopping(monitor="loss", patience=200, mode="min")]
      )
    with tpu_strategy.scope():
      model.save("/content/drive/MyDrive/Colab/models/transformer.keras")

print(history.history)


In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Colab/models/transformer.keras")
model.summary()

In [ ]:
# Graph history
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()